In [6]:
import os
import cv2
import json
import torch
import numpy as np
from tqdm import tqdm
from datetime import datetime
from segment_anything import sam_model_registry, SamPredictor
from pycocotools import mask as mask_utils

In [10]:
image_folder = r"C:\Users\Home\Sidewalk\IMAGE_FOLDER"
checkpoint_path = r"C:\Users\Home\Sidewalk\PATH_TO_MODEL\sam_vit_h_4b8939.pth"
model_type = "vit_h"
output_root = "segmentation_outputs"
os.makedirs(output_root, exist_ok=True)

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
sam.to(device)
predictor = SamPredictor(sam)

In [12]:
def binary_mask_to_rle(mask):
    rle = mask_utils.encode(np.asfortranarray(mask.astype(np.uint8)))
    rle["counts"] = rle["counts"].decode("utf-8")  # bytes to str for JSON
    return rle


annotation_id = 1
image_id = 1

In [ ]:
# --- Process each image ---
for fname in tqdm(os.listdir(image_folder)):
    if not fname.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    image_path = os.path.join(image_folder, fname)
    image_bgr = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    h, w = image_rgb.shape[:2]

    # Set up predictor
    predictor.set_image(image_rgb)
    points = []
    labels = []

    def click_event(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append([x, y])
            labels.append(1)
            cv2.circle(image_bgr, (x, y), 5, (0, 255, 0), -1)
            cv2.imshow("Click points, press 's' to segment", image_bgr)

    print(f"\nProcessing image: {fname}")
    cv2.imshow("Click points, press 's' to segment", image_bgr)
    cv2.setMouseCallback("Click points, press 's' to segment", click_event)

 50%|█████     | 1/2 [00:32<00:32, 32.94s/it]


Processing image: 001_png_jpg.rf.1eb25691beb17dfdbdf6ae9dcfb447f2.jpg


100%|██████████| 2/2 [01:05<00:00, 32.95s/it]


Processing image: 002_png_jpg.rf.5e353d1fd31ef6d233eddbbaae3d82e3.jpg


: 